In [ ]:
'''2do Parcial - Ejercicio2'''
'''Taboada Rodas Oscar'''

In [2]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize

%matplotlib inline


In [3]:
data = np.loadtxt(os.path.join('7. Internet_ads', 'Anuncios_internet.csv'), delimiter = ',')
# data = pd.read_csv('https://query.data.world/s/yzstkado34nyn736kvzkiyztebzyzt', low_memory=False)
data.shape

(500, 1559)

In [4]:
X = data[:, :1558]
y = data[:, 1558]

print('X \n',X,'\n y\n',y)


X 
 [[125.     125.       1.     ...   0.       0.       0.    ]
 [ 57.     468.       8.2105 ...   0.       0.       0.    ]
 [ 33.     230.       6.9696 ...   0.       0.       0.    ]
 ...
 [ 35.     126.       3.6    ...   0.       0.       0.    ]
 [ 32.      32.       1.     ...   0.       0.       0.    ]
 [ 75.     150.       2.     ...   0.       0.       0.    ]] 
 y
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0

In [5]:
X.shape[1]

1558

In [6]:
input_layer_size  = 1558
num_labels = 2
m = y.size

In [7]:
def calcularSigmoide(z):
    # Calcula la sigmoide de z.

    return 1.0 / (1.0 + np.exp(-z))

In [9]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size
    
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = calcularSigmoide(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea 
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [10]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate((np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0), axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 2

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [11]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 1.934819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.348558, 0.524722, 0.998003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [12]:
def oneVsAll(X, y, num_labels, lambda_):
    
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(calcularCosto, 
                                initial_theta, 
                                (X, (y == (c + 1)), lambda_), 
                                jac=True, 
                                method='BFGS',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [13]:
lambda_ = 0.01
all_theta = oneVsAll(X, y, num_labels, lambda_)

<ipython-input-9-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-9-d1ad24806257>:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [14]:
print(all_theta)

[[-2.18507446e+00 -2.28712065e-02  3.26275279e-02 ...  6.97847570e-02
  -9.90720817e-02  8.62273640e-03]
 [-2.90410520e-01 -6.66372667e-01 -8.38008507e-01 ... -4.99475274e-05
  -2.36593551e-05 -2.62881723e-06]]


In [15]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)
    return p + 1

In [16]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[195:205, :].copy()
yPrueba = y[195:205].copy()
# XPrueba = X[300:310, :].copy()
# yPrueba = y[300:310].copy()
print(XPrueba.shape)
print(len(XPrueba))
XPrueba = np.concatenate((np.ones((10, 1)), XPrueba), axis=1)
print(XPrueba)

p = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

(500, 1558)
Precision del conjuto de entrenamiento: 39.80%
(10, 1558)
10
[[  1.  60. 468. ...   0.   0.   0.]
 [  1.  60. 468. ...   0.   0.   0.]
 [  1.  73.  69. ...   0.   0.   0.]
 ...
 [  1.  12. 117. ...   0.   0.   0.]
 [  1. 110. 111. ...   0.   0.   0.]
 [  1. 161. 220. ...   0.   0.   0.]]
[1 1 1 1 1 1 1 1 1 1]
